In [1]:
import textworld
from textworld import GameMaker
from textworld.generator.data import KnowledgeBase
from textworld.generator.game import Event, Quest
from textworld.generator.game import GameOptions

# Make the generation process reproducible.
from textworld import g_rng  # Global random generator.
g_rng.set_seed(20180916)

In [2]:
from textworld.generator import compile_game
import io
import sys
import contextlib
import numpy as np

In [3]:
@contextlib.contextmanager
def capture_stdout():
    # Capture stdout.
    stdout_bak = sys.stdout
    sys.stdout = out = io.StringIO()
    try:
        yield out
    finally:
        # Restore stdout
        sys.stdout = stdout_bak


def _compile_test_game(game):
    grammar_flags = {
        "theme": "house",
        "include_adj": False,
        "only_last_action": True,
        "blend_instructions": True,
        "blend_descriptions": True,
        "refer_by_name_only": True,
        "instruction_extension": []
    }
    rng_grammar = np.random.RandomState(1234)
    grammar = textworld.generator.make_grammar(grammar_flags, rng=rng_grammar)
    game.change_grammar(grammar)

    game_file = textworld.generator.compile_game(game)
    return game_file



# Game Design - SuperHero

In [4]:

def build_and_compile_super_hero_game():
    M = GameMaker()
    roomA = M.new_room("Room A")
    alley = M.new_room("Alley")
    bank1 = M.new_room("Bank1")
    bank2 = M.new_room("Bank2")
    bank3 = M.new_room("Bank3")
    corridor = M.connect(roomA.east, alley.west)
    corridor1 = M.connect(alley.east, bank1.west)
    corridor1 = M.connect(alley.north, bank2.south)
    corridor1 = M.connect(alley.south, bank3.north)
    M.set_player(roomA)
    
    roomA.infos.desc = "You are in a road. Some mobs are planning to rob a bank. You need to stop them. Go east to the alley. You can find a person in the alley who has information about the roberry. Collect information from him and prevent the robbery."
    alley.infos.desc = "This is an alley. There is a person beside the table. He knows about the bank robbery."
    bank2.infos.desc = "This is the north bank. Some robbers are going to rob the bank. You can call the police and try to capture them or convince them to surrender. Or you can also shoot them to stop the robbery."
    
    money = M.new(type="o", name = 'money') 
    money.infos.desc = "it is money"
    M.inventory.add(money) 
    person = M.new(type="pr", name = "informant")
    person.infos.desc = "This person knows about the bank robbery. Do a favor for him. He will help you."
    M.add_fact("not_asked", person)
    M.add_fact("not_given", person)
    alley.add(person)
    
    robber = M.new(type="rbr", name = "robber")
    bank2.add(robber)
    M.add_fact("not_stopped", robber)
    M.add_fact("not_allowed", robber)
    M.add_fact("not_called", robber)
    
    
    # asking quest
    qst_event_asking = Event(conditions={M.new_fact("asked", person)})
    quest_asking = Quest(win_events=[qst_event_asking],
                      reward=2)
    # the wining quest
    qst_event_stopped_rob = Event(conditions={M.new_fact("asked", person),
                                             M.new_fact("stopped", robber)})
    win_quest = Quest(win_events=[qst_event_stopped_rob],
                      reward=2)

    # 1st failure condition
    failed_cmds1 = ["go east", "go south"]
    failed_event1 = M.new_event_using_commands(failed_cmds1)
    failed_quest_1 = Quest(win_events=[],
                           fail_events=[failed_event1])
    
    # 2nd failure condition
    failed_cmds2 = ["go east", "go east"]
    failed_event2 = M.new_event_using_commands(failed_cmds2)
    failed_quest_2 = Quest(win_events=[],
                           fail_events=[failed_event2])
    
    # 3rd failure condition
    failed_event3 = Event(conditions={
        M.new_fact("not_asked", person),
        M.new_fact("at", M._entities['P'], bank2)})
    
    failed_quest_3 = Quest(win_events=[],
                           fail_events=[failed_event3])
    
    
    failed_event4 = Event(conditions={
        M.new_fact("allowed", robber)})
    
    failed_quest_4 = Quest(win_events=[],
                   fail_events=[failed_event4])
    
    
    M.quests = [win_quest, quest_asking, failed_quest_1, failed_quest_2, failed_quest_3, failed_quest_4]
    game = M.build()

    game.main_quest = win_quest
    game_file = _compile_test_game(game)
    return game, game_file

# Game Design - Daily routine

In [5]:
def build_and_compile_daily_routine_game():
    M = GameMaker()
    classroom = M.new_room("Classroom")
    home = M.new_room("Home")
    carnival = M.new_room("carnival")
    clothes = M.new(type="cl", name = "clothes")
    
    corridor1 = M.connect(classroom.east, home.west)
    corridor2 = M.connect(home.east, carnival.west)

    M.set_player(classroom)
    
    classroom.infos.desc = "You are at your school. You have an assignment to submit today. Before going home, complete and submit the assignment."
    home.infos.desc = "You are at your home. A carnival is happening nearby your home. Your friends are there and they are expecting you. But before going to carnival clean up your room."
    carnival.infos.desc = "A large crowd has come to the carnival to enjoy it. Therefore there is a long line in the entrance."
    
    assignment = M.new(type="asgn", name = "assignment")
    line = M.new(type="ln", name = "line")
    M.add_fact("not_finished", assignment)
    M.add_fact("not_studied", assignment)
    classroom.add(assignment)

    M.add_fact("not_cleaned", clothes)
    M.add_fact("not_sorted", clothes)
    home.add(clothes)
    
    M.add_fact("not_joined", line)
    M.add_fact("not_checked", line)
    carnival.add(line)    
    
    qst_event_assignment1 = Event(conditions={M.new_fact("at", M._entities['P'], classroom),
                                        M.new_fact("finished", assignment)})
    quest_assignment1 = Quest(win_events=[qst_event_assignment1],
                      reward=2)
    
    qst_event_assignment2 = Event(conditions={M.new_fact("at", M._entities['P'], home),
                                        M.new_fact("cleaned", clothes)})
    quest_assignment2 = Quest(win_events=[qst_event_assignment2],
                      reward=2)
    
    qst_event_assignment3 = Event(conditions={M.new_fact("at", M._entities['P'], carnival),
                                        M.new_fact("joined", line)})
    quest_assignment3 = Quest(win_events=[qst_event_assignment3],
                      reward=2)
    
    qst_event_assignment4 = Event(conditions={M.new_fact("at", M._entities['P'], home),
                                        M.new_fact("finished", assignment)})
    quest_assignment4 = Quest(win_events=[qst_event_assignment4],
                      reward=1)
    
    qst_event_assignment5 = Event(conditions={M.new_fact("at", M._entities['P'], carnival),
                                        M.new_fact("cleaned", clothes)})
    quest_assignment5 = Quest(win_events=[qst_event_assignment5],
                      reward=1)
    
    failed_event = Event(conditions={
        M.new_fact("not_finished", assignment),
        M.new_fact("at", M._entities['P'], home)})
    
    failed_event2 = Event(conditions={
        M.new_fact("not_cleaned", clothes),
        M.new_fact("at", M._entities['P'], carnival)})
    
    failed_quest = Quest(win_events=[],
                           fail_events=[failed_event])
    
    failed_quest2 = Quest(win_events=[],
                           fail_events=[failed_event2])
    
    M.quests = [quest_assignment1, quest_assignment2, quest_assignment3, quest_assignment4, quest_assignment5, failed_quest, failed_quest2]
    game = M.build()

    game.main_quest = quest_assignment3
    game_file = _compile_test_game(game)
    return game, game_file

# Game Desing - HungryKid

In [6]:
def build_and_compile_hungrykid_game():
    M = GameMaker()
    home = M.new_room("Home")
    alley = M.new_room("Alley")
    downtown = M.new_room("Downtown")
    store = M.new_room("Store")
    
    corridor1 = M.connect(home.east, alley.west)
    corridor2 = M.connect(alley.east, store.west)
    corridor3 = M.connect(alley.north, downtown.south)
    M.set_player(home)
    
    passerby = M.new(type="pp", name = "passerby")
    people = M.new(type="ppl", name = "people")
    guard = M.new(type="gr", name = "guard")
    M.add_fact("not_killed", guard)
    M.add_fact("not_broken", guard)
    M.add_fact("not_stolen", guard)
    alley.add(passerby)
    store.add(guard)
    downtown.add(people)
    
    qst_event_assignment = Event(conditions={M.new_fact("at", M._entities['P'], store),
                                        M.new_fact("killed", guard),
                                            M.new_fact("broken", guard),
                                            M.new_fact("stolen", guard)})
    quest_assignment = Quest(win_events=[qst_event_assignment],
                      reward=5)

    M.quests = [quest_assignment]
    game = M.build()

    game.main_quest = quest_assignment
    game_file = _compile_test_game(game)
    return game, game_file

In [7]:
game, game_file = build_and_compile_hungrykid_game()

In [8]:
game_file

'./tw_games/252a7923-1028-49ed-ba23-cee9824bebb5.ulx'

In [9]:
import textworld
import textworld.gym
import gym
request_infos = textworld.EnvInfos(description=True, inventory=True, admissible_commands=True,won=True, lost=True, facts=True)

# Requesting additional information should be done when registering the game.
env_id = textworld.gym.register_game(game_file, request_infos)
env = gym.make(env_id)

obs, infos = env.reset()

In [ ]:
try:
    done = False
    obs, _ = env.reset()
    print(obs)
    nb_moves = 0
    while not done:
        print("infos: ",infos["admissible_commands"])
        command = input("> ")
        obs, score, done, infos = env.step(command)
        
        print(obs)
        nb_moves += 1

except KeyboardInterrupt:
    pass  # Press the stop button in the toolbar to quit the game.

print("Played {} steps, scoring {} points.".format(nb_moves, score))